# About Notebook

This notebook is an example of how to perform classification or regression predictions using images, in this case, x-ray images.

The intent of the code below is to encourage developers to build their own Convolutional Neural Network (CNN), test pre-existing models, or even use features extracted from pre-existing models by inputting these into shallow models, such as Decision Trees. The idea here is to explore and potentially leverage the strengths of different model architectures in order to better understand and solve image-based machine learning tasks.

In [24]:
import os

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler

from PIL import Image

import tensorflow as tf
from keras.applications import ResNet50
from keras.layers import Dense, GlobalAveragePooling2D, Flatten, Conv2D, MaxPooling2D
from keras.models import Model, Sequential
from keras.losses import BinaryCrossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.imagenet_utils import preprocess_input

from tqdm import tqdm

devices = tf.config.list_physical_devices()
for device in devices:
    print(device)

PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')


In [25]:
current_dir = os.getcwd()

repo_dir = current_dir[:current_dir.find("/notebooks")]

data_dir = os.path.join(repo_dir, "data")

kaggle_dir = os.path.join(data_dir, "kaggle")
kaggle_dir = os.path.join(kaggle_dir, "kaggle")

train_dir = os.path.join(kaggle_dir, "train")
test_dir = os.path.join(kaggle_dir, "test")

print(kaggle_dir)
print(train_dir)
print(test_dir, "\n")

/home/vinicius/repositories/x-ray-predict/data/kaggle/kaggle
/home/vinicius/repositories/x-ray-predict/data/kaggle/kaggle/train
/home/vinicius/repositories/x-ray-predict/data/kaggle/kaggle/test 



In [26]:
def get_images_path(directory, extension: str = ".png") -> list:
    """
    This function returns a list of all file paths that match a given file extension within a directory and its subdirectories.
    
    Parameters
    ----------
    directory : str
        The path of the directory to be searched. Subdirectories will be included in the search.

    extension : str, optional
        The file extension of the image files to be searched. Default is ".png".

    Returns
    -------
    list
        A sorted list of file paths of the images found within the directory and its subdirectories that match the specified file extension.

    Examples
    --------
    >>> get_images_path("/path/to/your/directory", ".jpg")
    ['/path/to/your/directory/image1.jpg', '/path/to/your/directory/subdirectory/image2.jpg', ...]
    """
    file_images = [
        os.path.join(dirpath, filename)
        for dirpath, dirnames, filenames in os.walk(directory)
        for filename in filenames
        if filename.endswith(extension)
    ]

    return sorted(file_images)


train_images_path = get_images_path(train_dir)
test_images_path = get_images_path(test_dir)

print("Train Images : ", len(train_images_path))
print("Test Images  : ", len(test_images_path))

Train Images :  10702
Test Images  :  11747


In [27]:
train_gender_df = pd.read_csv(os.path.join(data_dir, "train_gender.csv"))

train_age_df = pd.read_csv(os.path.join(data_dir, "train_age.csv"))

train_df = pd.merge(left=train_gender_df, right=train_age_df, how="inner", on="imageId")

train_df["directory"] = train_images_path

train_df["filename"] = train_df["directory"].apply(lambda x: x[-10:])

train_df.head()

,imageId,gender,age,directory,filename
0,0,0,89.0,/home/vinicius/repositories/x-ray-predict/data...,000000.png
1,1,0,72.0,/home/vinicius/repositories/x-ray-predict/data...,000001.png
2,2,1,25.0,/home/vinicius/repositories/x-ray-predict/data...,000002.png
3,3,1,68.0,/home/vinicius/repositories/x-ray-predict/data...,000003.png
4,4,0,37.0,/home/vinicius/repositories/x-ray-predict/data...,000004.png


In [28]:
train_df, val_df = train_test_split(
    train_df,
    test_size=0.3,
    random_state=42,
)

print("Train Dataset:       ", train_df.shape[0])
print("Validation Dataset:  ", val_df.shape[0])

Train Dataset:        7491
Validation Dataset:   3211


In [29]:
def load_images(df: pd.DataFrame, img_size: tuple) -> np.array:
    """Loads images and preprocesses them for the model."""
    image_list = []
    for filename in tqdm(df["directory"]):
        # Open image and resize
        img = Image.open(filename).convert("RGB").resize(img_size)
        # Convert image to array and preprocess
        img_array = np.array(img)
        img_array = preprocess_input(img_array)
        image_list.append(img_array)

    images = np.array(image_list)
    return images

In [30]:
image_size = (64, 64)

x_train, x_val = load_images(train_df, image_size), load_images(val_df, image_size)
y_gender_train, y_gender_val = train_df["gender"].values, val_df["gender"].values
y_age_train, y_age_val = train_df["age"].values, val_df["age"].values

100%|██████████| 3211/3211 [01:30<00:00, 35.41it/s]


In [31]:
x_train_normalized = x_train / 255
x_val_normalized = x_val / 255

# 1. Build own CNN

In [32]:
class WallNetClassifier(Sequential):
    def __init__(self):
        super().__init__()

        # Adding the first convolutional layer
        self.add(Conv2D(32, (3, 3), activation="relu", input_shape=(64, 64, 3)))

        # Adding a pooling layer to reduce dimensionality
        self.add(MaxPooling2D((2, 2)))

        # Adding a second convolutional layer
        self.add(Conv2D(64, (3, 3), activation="relu"))

        # Adding another pooling layer
        self.add(MaxPooling2D((2, 2)))

        # Adding a third convolutional layer
        self.add(Conv2D(64, (3, 3), activation="relu"))

        # Adding a Flatten layer to transform the feature matrix into a vector
        self.add(Flatten())

        # Adding a dense layer (or 'fully connected' layer)
        self.add(Dense(64, activation="relu"))

        # Adding the output layer
        self.add(Dense(1, activation="sigmoid"))

        # Compiling the model
        self.compile(
            optimizer="adam",
            loss=BinaryCrossentropy(),
            metrics=["accuracy"],
        )

    def train_gender_model(
        self,
        X_train: np.array,
        y_train: np.array,
        X_val: np.array,
        y_val: np.array,
        epochs: int = 10,
        batch_size: int = 32,
        verbose: int = 1,
    ):
        # Fitting the model
        self.fit(
            X_train,
            y_train,
            validation_data=(X_val, y_val),
            epochs=epochs,
            batch_size=batch_size,
            verbose=verbose,
        )
        
    def predict_gender(self, X: np.array):
        
        predictions = self.predict(X)

        predicted_classes = (predictions > 0.5).astype(int)

        return predicted_classes
    



class WallNetRegression(Sequential):
    def __init__(self):
        super().__init__()

        # Adding the first convolutional layer
        self.add(Conv2D(32, (3, 3), activation="relu", input_shape=(64, 64, 3)))

        # Adding a pooling layer to reduce dimensionality
        self.add(MaxPooling2D((2, 2)))

        # Adding a second convolutional layer
        self.add(Conv2D(64, (3, 3), activation="relu"))

        # Adding another pooling layer
        self.add(MaxPooling2D((2, 2)))

        # Adding a third convolutional layer
        self.add(Conv2D(64, (3, 3), activation="relu"))

        # Adding a Flatten layer to transform the feature matrix into a vector
        self.add(Flatten())

        # Adding a dense layer (or 'fully connected' layer)
        self.add(Dense(64, activation="relu"))

        # Adding the output layer
        # Linear activation for regression
        self.add(Dense(1, activation="linear"))  

        # Compiling the model
        self.compile(
            optimizer="adam",
            loss="mse",
            metrics=["mae"]
        )
        
    def train_age(
        self,
        X_train: np.array,
        y_train: np.array,
        X_val: np.array,
        y_val: np.array,
        epochs: int = 10,
        batch_size: int = 32,
        verbose: int = 1,
    ):
        # Fitting the model
        self.fit(
            X_train,
            y_train,
            validation_data=(X_val, y_val),
            epochs=epochs,
            batch_size=batch_size,
            verbose=verbose,
        )

    def predict_age(self, X: np.array):
        # Predicting age
        return self.predict(X)
    
wallnet_classifier = WallNetClassifier()
print(wallnet_classifier.summary())

print("\n\n\n")
    
wallnet_regression = WallNetRegression()
print(wallnet_regression.summary())

Model: "wall_net_classifier"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 64)        36928     
                                                                 
 flatten (Flatten)           (None, 9216)      

## Gender

In [33]:
# fit model gender
wallnet_classifier.train_gender_model(X_train=x_train_normalized, y_train=y_gender_train, X_val=x_val_normalized, y_val=y_gender_val)

Epoch 1/10


2023-06-27 19:50:08.614424: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 368197632 exceeds 10% of free system memory.


234/235 [============================>.] - ETA: 0s - loss: 0.4118 - accuracy: 0.8180

2023-06-27 19:50:27.518857: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 157827072 exceeds 10% of free system memory.


235/235 [==============================] - 21s 85ms/step - loss: 0.4116 - accuracy: 0.8180 - val_loss: 0.3442 - val_accuracy: 0.8564
Epoch 2/10
235/235 [==============================] - 22s 92ms/step - loss: 0.2576 - accuracy: 0.8980 - val_loss: 0.2463 - val_accuracy: 0.8950
Epoch 3/10
235/235 [==============================] - 19s 80ms/step - loss: 0.1905 - accuracy: 0.9286 - val_loss: 0.1946 - val_accuracy: 0.9290
Epoch 4/10
235/235 [==============================] - 18s 79ms/step - loss: 0.1589 - accuracy: 0.9389 - val_loss: 0.2023 - val_accuracy: 0.9321
Epoch 5/10
235/235 [==============================] - 20s 83ms/step - loss: 0.1321 - accuracy: 0.9519 - val_loss: 0.1780 - val_accuracy: 0.9383
Epoch 6/10
235/235 [==============================] - 20s 86ms/step - loss: 0.1119 - accuracy: 0.9602 - val_loss: 0.1789 - val_accuracy: 0.9377
Epoch 7/10
235/235 [==============================] - 19s 81ms/step - loss: 0.0930 - accuracy: 0.9668 - val_loss: 0.2058 - val_accuracy: 0.9402
Epo

In [34]:
# predict with validation images
y_pred = wallnet_classifier.predict_gender(x_val_normalized)

# Confusion Matrix
cm = confusion_matrix(y_gender_val, y_pred)

# Report Confusion Matrix
report = classification_report(y_gender_val, y_pred)

print("\n")
print(cm, "\n\n")
print(report)

  5/101 [>.............................] - ETA: 1s 

2023-06-27 19:53:24.168193: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 157827072 exceeds 10% of free system memory.


101/101 [==============================] - 2s 15ms/step


[[1790   96]
 [  81 1244]] 


              precision    recall  f1-score   support

           0       0.96      0.95      0.95      1886
           1       0.93      0.94      0.93      1325

    accuracy                           0.94      3211
   macro avg       0.94      0.94      0.94      3211
weighted avg       0.95      0.94      0.94      3211



## Age

In [35]:
# fit model age
wallnet_regression.train_age(X_train=x_train_normalized, y_train=y_age_train, X_val=x_val_normalized, y_val=y_age_val, epochs=10, batch_size=32)

Epoch 1/10


2023-06-27 19:53:25.999758: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 368197632 exceeds 10% of free system memory.


234/235 [============================>.] - ETA: 0s - loss: 435.9152 - mae: 16.8951

2023-06-27 19:53:44.198716: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 157827072 exceeds 10% of free system memory.


235/235 [==============================] - 20s 81ms/step - loss: 435.8469 - mae: 16.8944 - val_loss: 333.6973 - val_mae: 15.1526
Epoch 2/10
235/235 [==============================] - 20s 86ms/step - loss: 253.0375 - mae: 13.0141 - val_loss: 256.9884 - val_mae: 12.9510
Epoch 3/10
235/235 [==============================] - 20s 86ms/step - loss: 207.6963 - mae: 11.6391 - val_loss: 189.7469 - val_mae: 11.1897
Epoch 4/10
235/235 [==============================] - 18s 78ms/step - loss: 176.5767 - mae: 10.7421 - val_loss: 159.9057 - val_mae: 10.2216
Epoch 5/10
235/235 [==============================] - 19s 80ms/step - loss: 157.9096 - mae: 10.1033 - val_loss: 163.8023 - val_mae: 10.3232
Epoch 6/10
235/235 [==============================] - 21s 88ms/step - loss: 145.2508 - mae: 9.6478 - val_loss: 150.8340 - val_mae: 9.7713
Epoch 7/10
235/235 [==============================] - 20s 83ms/step - loss: 133.5338 - mae: 9.2056 - val_loss: 145.1154 - val_mae: 9.7205
Epoch 8/10
235/235 [===============

In [36]:
# predict age with validation images
y_pred = wallnet_regression.predict_age(x_val_normalized)

rmse = mean_squared_error(y_age_val, y_pred, squared=False).round(2)
mae = mean_absolute_error(y_age_val, y_pred).round(2)
r2 = r2_score(y_age_val, y_pred).round(2)

print("\n============ Regression model report ============")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"R^2 Score: {r2}")
print("=================================================")

101/101 [==============================] - 2s 16ms/step

============ Regression model report ============
Root Mean Squared Error (RMSE): 11.97
Mean Absolute Error (MAE): 9.47
R^2 Score: 0.6


# 2. ResNet50 - FineTunning
Using ResNet50, simply change the specific layer to suit the classification or regression problem.

In [37]:
class ResNet50Tunning:
    def __init__(self):
        self.base_model = ResNet50(
            weights="imagenet", include_top=False, input_shape=(64, 64, 3)
        )
        self.gender_model = None
        self.age_model = None

    def train_gender_model(
        self, X_train: np.array, y_train: np.array, X_val: np.array, y_val: np.array
    ):
        # Freeze the base layers
        for layer in self.base_model.layers:
            layer.trainable = False

        # Add custom layers for gender classification
        pooled_output = GlobalAveragePooling2D()(self.base_model.output)
        gender_output = Dense(1, activation="sigmoid")(pooled_output)
        self.gender_model = Model(inputs=self.base_model.input, outputs=gender_output)

        # Compile and train the gender classification model
        self.gender_model.compile(
            optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"]
        )

        self.gender_model.fit(
            X_train,
            y_train,
            validation_data=(X_val, y_val),
            epochs=10,
            batch_size=32,
        )

        return self.gender_model

    def train_age_model(
        self, X_train: np.array, y_train: np.array, X_val: np.array, y_val: np.array
    ):
        # Freeze the base layers
        for layer in self.base_model.layers:
            layer.trainable = False

        # Add custom layers for age regression
        pooled_output = GlobalAveragePooling2D()(self.base_model.output)
        age_output = Dense(1, activation="linear")(pooled_output)
        self.age_model = Model(inputs=self.base_model.input, outputs=age_output)

        # Compile and train the age regression model
        self.age_model.compile(optimizer="adam", loss="mse", metrics=["mae"])

        self.age_model.fit(
            X_train,
            y_train,
            validation_data=(X_val, y_val),
            epochs=10,
            batch_size=32,
        )

        return self.age_model
    
fine_tunning = ResNet50Tunning()

## Gender

In [38]:
gender_model = fine_tunning.train_gender_model(X_train=x_train_normalized, y_train=y_gender_train, X_val=x_val_normalized, y_val=y_gender_val)

Epoch 1/10
235/235 [==============================] - 55s 225ms/step - loss: 0.6612 - accuracy: 0.5985 - val_loss: 0.6439 - val_accuracy: 0.6191
Epoch 2/10
235/235 [==============================] - 52s 220ms/step - loss: 0.6307 - accuracy: 0.6464 - val_loss: 0.6270 - val_accuracy: 0.6609
Epoch 3/10
235/235 [==============================] - 50s 211ms/step - loss: 0.6130 - accuracy: 0.6695 - val_loss: 0.6146 - val_accuracy: 0.6702
Epoch 4/10
235/235 [==============================] - 50s 215ms/step - loss: 0.6007 - accuracy: 0.6852 - val_loss: 0.6063 - val_accuracy: 0.6802
Epoch 5/10
235/235 [==============================] - 54s 229ms/step - loss: 0.5911 - accuracy: 0.6918 - val_loss: 0.6020 - val_accuracy: 0.6724
Epoch 6/10
235/235 [==============================] - 55s 235ms/step - loss: 0.5840 - accuracy: 0.6987 - val_loss: 0.5940 - val_accuracy: 0.6836
Epoch 7/10
235/235 [==============================] - 69s 295ms/step - loss: 0.5791 - accuracy: 0.7052 - val_loss: 0.5893 - val_ac

In [39]:
# predict with validation images
predictions = gender_model.predict(x_val_normalized)

y_pred = (predictions > 0.5).astype(int)

# Confusion Matrix
cm = confusion_matrix(y_gender_val, y_pred)

# Report Confusion Matrix
report = classification_report(y_gender_val, y_pred)

print("\n")
print(cm, "\n\n")
print(report)

101/101 [==============================] - 17s 164ms/step


[[1608  278]
 [ 661  664]] 


              precision    recall  f1-score   support

           0       0.71      0.85      0.77      1886
           1       0.70      0.50      0.59      1325

    accuracy                           0.71      3211
   macro avg       0.71      0.68      0.68      3211
weighted avg       0.71      0.71      0.70      3211



## Age

In [40]:
age_model = fine_tunning.train_age_model(X_train=x_train_normalized, y_train=y_age_train, X_val=x_val_normalized, y_val=y_age_val)

Epoch 1/10
235/235 [==============================] - 53s 216ms/step - loss: 2165.5774 - mae: 42.2636 - val_loss: 1475.3674 - val_mae: 33.3562
Epoch 2/10
235/235 [==============================] - 49s 208ms/step - loss: 1085.6484 - mae: 27.2774 - val_loss: 758.8674 - val_mae: 21.8663
Epoch 3/10
235/235 [==============================] - 52s 223ms/step - loss: 598.9350 - mae: 19.5086 - val_loss: 473.7922 - val_mae: 17.6046
Epoch 4/10
235/235 [==============================] - 56s 240ms/step - loss: 421.3228 - mae: 17.1070 - val_loss: 386.4418 - val_mae: 16.7223
Epoch 5/10
235/235 [==============================] - 54s 231ms/step - loss: 371.4050 - mae: 16.6664 - val_loss: 366.5685 - val_mae: 16.6462
Epoch 6/10
235/235 [==============================] - 55s 233ms/step - loss: 361.3814 - mae: 16.6525 - val_loss: 363.7717 - val_mae: 16.6937
Epoch 7/10
235/235 [==============================] - 58s 247ms/step - loss: 359.5862 - mae: 16.6747 - val_loss: 363.4723 - val_mae: 16.7289
Epoch 8/10

In [41]:
# predict age with validation images
y_pred = age_model.predict(x_val_normalized)

rmse = mean_squared_error(y_age_val, y_pred, squared=False).round(2)
mae = mean_absolute_error(y_age_val, y_pred).round(2)
r2 = r2_score(y_age_val, y_pred).round(2)

print("\n============ Regression model report ============")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"R^2 Score: {r2}")
print("=================================================")

101/101 [==============================] - 17s 162ms/step

============ Regression model report ============
Root Mean Squared Error (RMSE): 19.04
Mean Absolute Error (MAE): 16.71
R^2 Score: -0.0


# 3. ResNet50 - Transfer Learning
Extract features to DecisionTree Model

In [42]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(64, 64, 3))
model = Model(inputs=base_model.input, outputs=base_model.output)

features_train = model.predict(x_train_normalized)
features_val = model.predict(x_val_normalized)

features_train = features_train.reshape(features_train.shape[0], -1)
features_val = features_val.reshape(features_val.shape[0], -1)

101/101 [==============================] - 15s 145ms/step


In [43]:
scaler = StandardScaler()

features_train_standardize = scaler.fit_transform(features_train)
features_val_standardize = scaler.transform(features_val)

## Gender

In [44]:
clf = DecisionTreeClassifier()

clf.fit(features_train_standardize, y_gender_train)

DecisionTreeClassifier()

In [45]:
y_pred = clf.predict(features_val_standardize)

In [46]:
# Confusion Matrix
cm = confusion_matrix(y_gender_val, y_pred)

# Report Confusion Matrix
report = classification_report(y_gender_val, y_pred)

print("\n")
print(cm, "\n\n")
print(report)



[[1384  502]
 [ 460  865]] 


              precision    recall  f1-score   support

           0       0.75      0.73      0.74      1886
           1       0.63      0.65      0.64      1325

    accuracy                           0.70      3211
   macro avg       0.69      0.69      0.69      3211
weighted avg       0.70      0.70      0.70      3211



## Age

In [47]:
clf = DecisionTreeRegressor()

clf.fit(features_train_standardize, y_age_train)

DecisionTreeRegressor()

In [48]:
y_pred = clf.predict(features_val_standardize)

In [49]:
rmse = mean_squared_error(y_age_val, y_pred, squared=False).round(2)
mae = mean_absolute_error(y_age_val, y_pred).round(2)
r2 = r2_score(y_age_val, y_pred).round(2)

print("\n============ Regression model report ============")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"R^2 Score: {r2}")
print("=================================================")


============ Regression model report ============
Root Mean Squared Error (RMSE): 23.29
Mean Absolute Error (MAE): 18.2
R^2 Score: -0.5


This is the worst possible outcome. It is evident that extracting features through ResNet50 and feeding them to a Decision Tree Regressor model isn't the best scenario. The model fails to generalize for validation data, resulting in a negative R-squared value and high Mean Absolute Error (MAE). This suggests that the model performs worse than a hypothetical model that always predicts the mean of the target variable. The negative R-squared indicates that the model cannot account for the variability in the response variable around its mean, and the high MAE shows that our predictions are, on average, far from the actual values.

# 4. Submit Result

The technique that yielded the best result was the creation of our own Convolutional Neural Network, 'WallNet'.

It demonstrated an accuracy of 94% for gender classification and had the best evaluation metrics for age regression of the patients. Given this scenario, we can submit our final result.

* The evaluation metric for gender competition is the area under the receiver operating characteristic curve (AUC)
* The evaluation metric for age competition is the mean absolute error (MAE)

In [51]:
test_gender_df = pd.read_csv(os.path.join(data_dir, "sample_submission_gender.csv"))
test_age_df = pd.read_csv(os.path.join(data_dir, "sample_submission_age.csv"))
test_df = pd.merge(left=test_gender_df, right=test_age_df, how="inner", on="imageId")

test_df["directory"] = test_images_path
test_df["filename"] = test_df["directory"].apply(lambda x: x[-10:])

test_df.head()

,imageId,gender,age,directory,filename
0,0,0.5,0,/home/vinicius/repositories/x-ray-predict/data...,000000.png
1,1,0.5,0,/home/vinicius/repositories/x-ray-predict/data...,000001.png
2,2,0.5,0,/home/vinicius/repositories/x-ray-predict/data...,000002.png
3,3,0.5,0,/home/vinicius/repositories/x-ray-predict/data...,000003.png
4,4,0.5,0,/home/vinicius/repositories/x-ray-predict/data...,000004.png


In [52]:
image_size = (64, 64)
x_test = load_images(test_df, image_size)
x_test_normalized = x_test / 255

100%|██████████| 11747/11747 [04:37<00:00, 42.31it/s]


## Gender

In [53]:
gender_submission_df = test_df.loc[:, ["imageId", "gender"]].copy()

gender_submission_df["gender"] = wallnet_classifier.predict(x_test_normalized)

gender_submission_df.to_csv("sample_submission_gender.csv", index=False, sep=",", decimal=".")

gender_submission_df.head()

368/368 [==============================] - 7s 18ms/step


,imageId,gender
0,0,0.895959
1,1,0.005626
2,2,0.999929
3,3,0.021404
4,4,0.997383


## Age

In [54]:
age_submission_df = test_df.loc[:, ["imageId", "age"]].copy()

age_submission_df["age"] = wallnet_regression.predict(x_test_normalized).astype(int)

age_submission_df.to_csv("sample_submission_age.csv", index=False, sep=",", decimal=".")

age_submission_df.head()

368/368 [==============================] - 6s 17ms/step


,imageId,age
0,0,49
1,1,37
2,2,41
3,3,68
4,4,51
